# Data Handling

In this notebook we write all of the data handling relevant functions: <br>
#### read_sumo_net(filename)
    1. input: The path to the SUMO file with the city graph
    2. output: The city's graph in a networkx graph object
#### add_edge_features_from_xml(G, xml_filename, interval_begin)
    1. inputs: The networkx graph object of the city; The path xml file with the traffic data; The beginning of the interval we want to work with
    2. output: The same networkx graph object updated with the sumo graph features (traffic volumes, etc)
#### nx_to_pyg(graph)
    1. input: The networkx graph object
    2. output: The same graph converted into a pytorch graph object
#### data_preparation(data, edges_to_hide, constant_normalization)
    1. input: data: The list of pytorch graph objects with the right edge and node features; The list of edges we want to cover, a boolean, which determines wether we work with the maximum edge feature or a constant normalization.
    2. output: The prepared training data, which have -1 as edge features for unknown edges, and normalized edge features for the known. The prepared test data, which have all the edge features normalized. The sum of the squares of all the normalized edge features for benchmarking the model's performance.


In [3]:
#Necessary imports
import sumolib
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import random

import torch
import torch_geometric
import torch_geometric.data as Data
import torch_geometric.utils as pyg_utils

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import numpy as np
import json
import sys

In [15]:
#Function to read in the network we work with into a networkx object with the nodes and edges, no features yet
def read_sumo_net(filename):
    net = sumolib.net.readNet(filename)
    G = nx.Graph()
    # Add nodes
    for node in net.getNodes():
        G.add_node(node.getID(), pos=(node.getCoord()))
    # Add edges
    for edge in net.getEdges():
        for lane in edge.getLanes():
            speed = lane.getSpeed()
            length = lane.getLength()
            average_travelling_time = length/speed
            edge_id = lane.getEdge().getID()
            starting_node_id = net.getEdge(edge_id).getFromNode().getID()
            ending_node_id = net.getEdge(edge_id).getToNode().getID()
            G.add_edge(starting_node_id, ending_node_id, edge_id = edge_id, average_travelling_time = average_travelling_time)
    return G

In [3]:
#Function to add the features to the network graph we created already
def add_edge_features_from_xml(G, xml_filename, interval_begin):
    # Parse the XML file
    tree = ET.parse(xml_filename)
    root = tree.getroot()
    #Find the interval corresponding to the interval_begin time 
    interval = root.find(f'.//interval[@begin="{interval_begin}"]')
    #Extract all the features of the edges
    edges = interval.findall('.//edge')
    for edge in edges:
        edge_id = edge.get('id')
        edge_features = {}
        edge_features['left'] = edge.get('left')
        #We can add other features here
        #Iterate through the edges in the existing NetworkX graph
        for xml_edge_id, xml_edge_data in G.edges.items():
            if G.get_edge_data(xml_edge_id[0],xml_edge_id[1])['edge_id'] == edge_id:
                G.edges[xml_edge_id].update(edge_features)
    return G

In [5]:
#Function to convert the networkx graph into a pytorch graph object
def nx_to_pyg(graph):
    # Convert NetworkX graph to PyTorch Geometric Data object
    pyg_data = Data.Data()
    #We have to number the nodes, because that is how Data object works
    # Mapping between string node IDs and numerical indices
    node_id_to_index = {node_id: i for i, node_id in enumerate(graph.nodes)}

    # Set node features
    num_nodes = graph.number_of_nodes()
    node_features = np.zeros((num_nodes, 2))  # Assuming num_features is known, this is important to change, if we want to change something, altough I do not think that will be the case for us
    for i, (node, features) in enumerate(graph.nodes(data=True)):
        node_features[i] = [features['pos'][0], features['pos'][1]]  # Add node features accordingly, this case the coordinates
    pyg_data.x = torch.tensor(node_features, dtype=torch.float)

    # Set edge features and edge indices
    edge_index = []
    edge_features = []
    for u, v, features in graph.edges(data=True):
        # Map string node IDs to numerical indices
        u_index = node_id_to_index[u]
        v_index = node_id_to_index[v]
        edge_index.append([u_index, v_index])
        edge_features.append([float(features['left'])])  # Add edge features accordingly, if we add more features, we have to change this line

    pyg_data.edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    pyg_data.edge_attr = torch.tensor(edge_features, dtype=torch.float)

    return pyg_data

In [6]:
#Function to prepare the data (list of pytorch graph objects with the correct node and edge features)
def data_preparation(data, edges_to_hide, constant_normalization):
    prepared_training_data = []
    prepared_test_data = []
    summa = 0
    for graph in data:
        edge_features = graph.edge_attr
        if constant_normalization:
            max_value = edge_features.max()
        else:
            max_value = (2*1800)/4 #In Győr for the majority of the city there's only 2 lanes on the busiest road
        normalized_edge_features = edge_features / max_value
        graph.edge_attr = normalized_edge_features
        graph_test = graph.clone()
        edge_features_to_sum = graph_test.edge_attr
        squared_edge_features = edge_features_to_sum**2
        summa += squared_edge_features.mean()
        graph.edge_attr[edges_to_hide] = -1
        prepared_training_data.append(graph)
        prepared_test_data.append(graph_test)
    return prepared_training_data, prepared_test_data, summa/len(data)